In [244]:
import csv
import pandas as pd
from pandas import DataFrame
import datetime
import math

In [245]:
df=pd.read_csv("tasks.csv",parse_dates=True)
print(df.head())

    Task   hour
0  Task1  10.00
1  Task2   1.00
2  Task3   0.50
3  Task4   0.75


In [246]:
total_work=0.0
for i in range(len(df)):
    total_work+=df.iloc[i].hour
    #print(df.iloc[i].hour)
print(total_work)

12.25


In [247]:
df1=pd.read_csv("predictedValue.csv",parse_dates=True)
print(df1.head())

   Free  hour  minu  weekday
0     1     0    15        3
1     1     0    30        3
2     1     0    45        3
3     1     1     0        3
4     1     1    15        3


In [248]:
total_free={
    0:0,
    1:0,
    2:0,
    3:0,
    4:0,
    5:0,
    6:0
}
for i in range(len(df1)):
    #total_work+=df.iloc[i].hour
    #print(df1.iloc[i].hour)
    #print(df1.iloc[i].weekday)
    weekday=df1.iloc[i].weekday
    if(df1.iloc[i].Free):
        total_free[weekday]+=0.25


print(total_free)

{0: 7.25, 1: 13.25, 2: 13.0, 3: 11.5, 4: 10.75, 5: 11.0, 6: 9.25}


In [249]:
Free_Time=0
for i in range(len(total_free)):
    Free_Time+=total_free[i]
print(Free_Time)

76.0


In [250]:
extra_time=math.floor((Free_Time-total_work)/7)
print(extra_time)

9


In [251]:
df3=df.copy()
print(df3.head())
df4 = pd.DataFrame(columns=['name','hour','weekday'])
#for i in range(len(total_free)):
for i in range(len(total_free)):
    rem_time=total_free[i]-extra_time
    if(rem_time>0):
        print(i)
        j=0
        
        while (rem_time!=0.0 and j<len(df3)):

            temp=min(df3.iloc[j].hour,rem_time)
            if(temp>0):
                rem_time=rem_time-temp
                      
                df3.loc[j,"hour"]=df3.iloc[j].hour-temp
            

                df4 = df4.append({'name': df3.iloc[j].Task, 'hour': temp,'weekday':i}, ignore_index=True)
            
            
            j+=1
            
print(df4)
            
    

    Task   hour
0  Task1  10.00
1  Task2   1.00
2  Task3   0.50
3  Task4   0.75
1
2
3
4
5
6
    name  hour weekday
0  Task1  4.25       1
1  Task1  4.00       2
2  Task1  1.75       3
3  Task2  0.75       3
4  Task2  0.25       4
5  Task3  0.50       4
6  Task4  0.75       4


In [252]:
df5 = pd.DataFrame(columns=['starth','startm','endh','endm','total','weekday'])
flag=0

total=0.0


for i in range(len(df1)):
    if(df1.iloc[i].Free==1 and flag == 0):
        starth=df1.iloc[i].hour
        startm=df1.iloc[i].minu

        flag=1
    
    if(df1.iloc[i].Free==1 and flag == 1):
        total+=0.25

    try:    
        if((df1.iloc[i].Free==0 and flag == 1) or (df1.iloc[i].weekday != df1.iloc[i-1].weekday and  flag == 1)) :
            endh=df1.iloc[i].hour
            endm=df1.iloc[i].minu
            weekday=df1.iloc[i-1].weekday
            flag=0
            df5 = df5.append({'starth': starth,'startm': startm, 'endh':endh, 'endm':endm,'total': total,'weekday':weekday }, ignore_index=True)
            
            total=0
    except:
        pass
    
print(df5.head())

                                                   
                                                   
                                                
        
    
    

   starth  startm  endh  endm  total  weekday
0     0.0    15.0   1.0  45.0   1.50      3.0
1    11.0    30.0  13.0  45.0   2.25      3.0
2    14.0     0.0  14.0  15.0   0.25      3.0
3    16.0    15.0  17.0  15.0   1.00      3.0
4    17.0    30.0  18.0   0.0   0.50      3.0


In [253]:
df5=df5.sort_values(by=['weekday', 'total'], ascending=[1,0])

print(df5.head())
    

    starth  startm  endh  endm  total  weekday
31    19.0     0.0   0.0   0.0   5.25      0.0
28    13.0    45.0  14.0  30.0   0.75      0.0
29    15.0    30.0  16.0  15.0   0.75      0.0
30    16.0    30.0  17.0  15.0   0.75      0.0
36    17.0    30.0   0.0   0.0   6.75      1.0


In [254]:
df6 = pd.DataFrame(columns=['Task','start','end','weekday'])
for i in range(len(df4)):
    weekday=df4.iloc[i].weekday
    rem_time=df4.iloc[i].hour
    name=df4.iloc[i]['name']
    print(name)
    j=0
    #d=df5.query('weekday =='+str(weekday))
    
    while(rem_time!=0.0 and j<len(df5)):
        if(df5.iloc[j].weekday==weekday):
            temp=min(df5.iloc[j].total,rem_time)
            rem_time=rem_time-temp
            df5.iloc[j].total=df5.iloc[j].total-temp
            
            starth=df5.iloc[j].starth
            startm=df5.iloc[j].startm
            hour=int(temp)
            minu=temp-hour
            if(minu==0.25):
                minu=15
            if(minu == 0.5):
                minu=30
            if(minu == 0.75):
                minu=45
      
            endh=int(starth)+hour
            endm=int(startm)+minu
            if(endm>=60):
                endh+=1
                endm-=60
            
            df5.iloc[j].starth=endh
            df5.iloc[j].startm=endm
      
            
            
            start=str(int(starth))+" : "+str(int(startm))
            end=str(int(endh))+" : "+str(int(endm))
            df6 = df6.append({'Task':name,'start': start, 'end':end,'weekday':weekday }, ignore_index=True)
     
        j+=1
        
print(df6)


Task1
Task1
Task1
Task2
Task2
Task3
Task4
    Task    start      end weekday
0  Task1  17 : 30  21 : 45       1
1  Task1   11 : 0   15 : 0       2
2  Task1  18 : 15   20 : 0       3
3  Task2   20 : 0  20 : 45       3
4  Task2   19 : 0  19 : 15       4
5  Task3  19 : 15  19 : 45       4
6  Task4  19 : 45  20 : 30       4


In [255]:
df6.to_csv("scheduled_Task.csv",mode = 'w')